<a href="https://colab.research.google.com/github/FREDY129053/AI_Systems_LABS/blob/main/Lab_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лаба 3.5: Метод опорных векторов

## Получение данных

In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
banknote_authentication = fetch_ucirepo(id=267)

df = banknote_authentication.data.original
print(f"Размерность данных: {df.shape}")
df.head()

Размерность данных: (1372, 5)


,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


## Практические задания

### 1. Проанализировать разные результаты для набора данных BanknoteAuthentification, в чём разница базовых настроек алгоритма в разных инструментах?

In [5]:
df_copy = df.copy()

X = df_copy.drop('class', axis=1)
y = df_copy['class']
print(f"Размер X = {X.shape}")
print(f"Размер y = {y.shape}")

Размер X = (1372, 4)
Размер y = (1372,)


In [6]:
import pandas as pd

df_class_0 = df[df['class'] == 0]
df_class_1 = df[df['class'] == 1]

X_test_0 = df_class_0.tail(107).drop('class', axis=1)
y_test_0 = df_class_0.tail(107)['class']

X_test_1 = df_class_1.tail(119).drop('class', axis=1)
y_test_1 = df_class_1.tail(119)['class']

X_test = pd.concat([X_test_0, X_test_1])
y_test = pd.concat([y_test_0, y_test_1])

X_train = df.drop(X_test.index).drop('class', axis=1)
y_train = df.drop(X_test.index)['class']

print(f"Размер обучающей выборки X_train: {X_train.shape}")
print(f"Размер обучающей выборки y_train: {y_train.shape}")
print(f"Размер тестовой выборки X_test: {X_test.shape}")
print(f"Размер тестовой выборки y_test: {y_test.shape}")

Размер обучающей выборки X_train: (1146, 4)
Размер обучающей выборки y_train: (1146,)
Размер тестовой выборки X_test: (226, 4)
Размер тестовой выборки y_test: (226,)


#### SVC из Sklearn

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

classifier_sklearn = SVC(kernel='rbf', max_iter=200000, cache_size=8000, random_state=42)

classifier_sklearn.fit(X_train, y_train)

y_pred_sklearn = classifier_sklearn.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred_sklearn) * 100}%")

Accuracy = 99.5575221238938%


#### SVC PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

session = SparkSession.builder.appName("SVC").getOrCreate()

train_df_spark = session.createDataFrame(X_train.join(y_train))
test_df_spark = session.createDataFrame(X_test.join(y_test))

assembler = VectorAssembler(inputCols=X_train.columns.to_list(), outputCol="features")
train_df_spark = assembler.transform(train_df_spark)
test_df_spark = assembler.transform(test_df_spark)

svc = LinearSVC(maxIter=200000, maxBlockSizeInMB=8000, labelCol="class", featuresCol="features")
model_spark = svc.fit(train_df_spark)

pred = model_spark.transform(test_df_spark)
eval = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="accuracy")
print(f"PySpark accuracy = {eval.evaluate(pred) * 100}%")

session.stop()

PySpark accuracy = 99.11504424778761%


### 2. Найти наилучшие параметры для данных BanknoteAuthentification, используя технику кросс-валидации

In [8]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

model_accuracy = SVC(cache_size=8000, random_state=42)

param_grid = {
    "kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    "max_iter": [-1, 200000, 400000]
}

grid_search = GridSearchCV(
  estimator=model_accuracy,
  param_grid=param_grid,
  cv=StratifiedKFold(5),
  scoring='accuracy',
  n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best parameters for SVC found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Best parameters for SVC found:  {'kernel': 'rbf', 'max_iter': -1}
# Best cross-validation score:  0.9965065502183407

Best parameters for SVC found:  {'kernel': 'rbf', 'max_iter': -1}
Best cross-validation score:  0.9965065502183407


#### Best params by 'roc_auc'

In [9]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

model_roc_auc = SVC(cache_size=8000, random_state=42)

param_grid = {
    "kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    "max_iter": [-1, 200000, 400000]
}

grid_search = GridSearchCV(
  estimator=model_roc_auc,
  param_grid=param_grid,
  cv=StratifiedKFold(5),
  scoring='roc_auc',
  n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best parameters for SVC found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Best parameters for SVC found:  {'kernel': 'rbf', 'max_iter': -1}
# Best cross-validation score:  1.0

Best parameters for SVC found:  {'kernel': 'rbf', 'max_iter': -1}
Best cross-validation score:  1.0


### 3. Возможно ли улучшить точность алгоритма для данных AdultIncome, используя другие параметры (gamma, C, параметры связанные с SVM)? Подобрать параметры, дающие большую точность или показать, что для большого набора параметров точность улучшить не удаётся

In [10]:
df['class'].value_counts()

,count
class,
0,762
1,610


#### Tune 'accuracy' model

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

model_accuracy_full_tune = SVC(cache_size=8000, random_state=42)

param_grid = {
    "kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    "C": [0.5, 1.5, 2.0, 2.5],
    "gamma": ['scale', 'auto', 1.0, 1.5],
    "degree": [2, 3, 4],
    "max_iter": [-1, 200000, 400000],
    "class_weight": [None, 'balanced', {0: 762/df.shape[0], 1: 610/df.shape[0]}]
}

grid_search = GridSearchCV(
  estimator=model_accuracy_full_tune,
  param_grid=param_grid,
  cv=StratifiedKFold(5),
  scoring='accuracy',
  n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best parameters for SVC more params found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Best parameters for SVC more params found:  {'C': 0.5, 'class_weight': None, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1}
# Best cross-validation score:  1.0

Best parameters for SVC more params found:  {'C': 0.5, 'class_weight': None, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1}
Best cross-validation score:  1.0


#### Tune 'roc_auc' model

In [11]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

model_roc_auc_full_tune = SVC(cache_size=8000, random_state=42)

param_grid = {
    "kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    "C": [0.5, 1.5, 2.0, 2.5],
    "gamma": ['scale', 'auto', 1.0, 1.5],
    "degree": [2, 3, 4],
    "max_iter": [-1, 200000, 400000],
    "class_weight": [None, 'balanced', {0: 762/df.shape[0], 1: 610/df.shape[0]}]
}

grid_search = GridSearchCV(
  estimator=model_roc_auc_full_tune,
  param_grid=param_grid,
  cv=StratifiedKFold(5),
  scoring='roc_auc',
  n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best parameters for SVC more params found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Best parameters for SVC more params found:  {'C': 0.5, 'class_weight': None, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1}
# Best cross-validation score:  1.0
# gamma: 'scale' на тестовой выборке не предсказывает 1% класса '0' (одно значение примерно)

Best parameters for SVC more params found:  {'C': 0.5, 'class_weight': None, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1}
Best cross-validation score:  1.0


Подкрутив другие параметры можно добиться точности 100%)

In [28]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

classifier_sklearn2_accurcy = SVC(C=0.5, class_weight=None, degree=2, gamma='auto', kernel='rbf', max_iter=-1, cache_size=8000, random_state=42)

classifier_sklearn2_accurcy.fit(X_train, y_train)

y_pred_sklearn_accuracy = classifier_sklearn2_accurcy.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred_sklearn_accuracy) * 100}%")

Accuracy = 100.0%


In [29]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

classifier_sklearn2_roc_auc = SVC(C=0.5, class_weight=None, degree=2, gamma='auto', kernel='rbf', max_iter=-1, cache_size=8000, random_state=42)

classifier_sklearn2_roc_auc.fit(X_train, y_train)

y_pred_sklearn_roc_auc = classifier_sklearn2_roc_auc.predict(X_test)
print(f"ROC-AUC = {roc_auc_score(y_test, y_pred_sklearn_roc_auc) * 100}/1")

ROC-AUC = 100.0/1


In [25]:
# https://muthu.co/understanding-the-classification-report-in-sklearn/
from sklearn.metrics import classification_report

print('-------------------- Accuracy Report --------------------')
print(classification_report(y_test, y_pred_sklearn_accuracy))

print('\n-------------------- ROC-AUC Report --------------------')
print(classification_report(y_test, y_pred_sklearn_roc_auc))

-------------------- Accuracy Report --------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       107
           1       1.00      1.00      1.00       119

    accuracy                           1.00       226
   macro avg       1.00      1.00      1.00       226
weighted avg       1.00      1.00      1.00       226


-------------------- ROC-AUC Report --------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       107
           1       1.00      1.00      1.00       119

    accuracy                           1.00       226
   macro avg       1.00      1.00      1.00       226
weighted avg       1.00      1.00      1.00       226

